In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time
import pickle
import napari
import numpy as np
import pandas as pd

from skimage.io import imread

import btrack
from btrack.constants import BayesianUpdates

sys.path.append('../libraries')
import input_functions as inp_f

In [2]:
info_file_path = r'Z:\Garrett\Livecell\030124_Lorenzo_livecell_TBHP_Dose_Curve\Well_B03\down\data\info_B03.txt'

In [3]:
# read the file
info_file = open(info_file_path, 'r')
info_lines = info_file.readlines()
info_file.close()

# read info about the data frame
exp_dir,df_name = inp_f.read_df_info(info_lines)

df_dir = os.path.join(exp_dir,'df')
save_dir = df_dir

frames_to_exclude = inp_f.read_frames_2_exclude(info_lines)
#frames_to_exclude = eval(frames_to_exclude)

modelPath = os.path.join(exp_dir,'code','libraries','cell_config.json')

## Read in the data frame objects data

In [4]:
data_df = pd.read_pickle(os.path.join(df_dir,df_name))

In [5]:
len(data_df)

75384

In [6]:
# create a structure suitable for tracking

# choose objects 
sel_vector = [not(x in frames_to_exclude) for x in data_df.t]

objects_gen = data_df.loc[sel_vector,['label','area','centroid-1','centroid-0','major_axis_length','minor_axis_length','t']]

objects_gen.columns=['ID', 'area', 'x', 'y', 'major_axis_length','minor_axis_length','t']
objects_gen['z']=0
objects_gen['label']=5
objects_gen['prob']=0
objects_gen['dummy']=False
objects_gen['states']=0

#objects_gen.head()

In [7]:
len(objects_gen)

75384

## Tracking proper

In [8]:
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

    # configure the tracker using a config file
    tracker.configure_from_file(modelPath)
    
    # approximate
    tracker.update_method = BayesianUpdates.APPROXIMATE
    tracker.max_search_radius = 500

    # append the objects to be tracked
    tracker.append(objects_gen)

    # set the volume (Z axis volume is set very large for 2D data)
    tracker.volume=((0, data_df.size_x[0]), (0, data_df.size_y[0]), (-1e5, 1e5))

    # track them (in interactive mode)
    tracker.track_interactive(step_size=100)

    # generate hypotheses and run the global optimizer
    #tracker.optimize()

    # optional: get the data in a format for napari
    data, properties, graph = tracker.to_napari(ndim=2)
    # pickle Napari data
    with open(os.path.join(df_dir,'track.pkl'),'wb') as f:
        pickle.dump([data,properties,graph],f)

[INFO][2024/04/17 12:23:37 PM] Loaded btrack: C:\Users\gases\.conda\envs\livecell_tracking\Lib\site-packages\btrack\libs\libtracker.DLL
[INFO][2024/04/17 12:23:37 PM] Starting BayesianTracker session
[INFO][2024/04/17 12:23:37 PM] Loading configuration file: Z:\Garrett\Livecell\030124_Lorenzo_livecell_TBHP_Dose_Curve\Well_B03\down\code\libraries\cell_config.json
[INFO][2024/04/17 12:23:37 PM] Objects are of type: <class 'pandas.core.frame.DataFrame'>
[WARNING][2024/04/17 12:23:38 PM] `track_interactive` will be deprecated. Use `track` instead.
[INFO][2024/04/17 12:23:38 PM] Starting tracking... 
[INFO][2024/04/17 12:23:38 PM] Update using: ['MOTION']
[INFO][2024/04/17 12:23:38 PM] Tracking objects in frames 0 to 99 (of 240)...
[INFO][2024/04/17 12:23:39 PM]  - Timing (Bayesian updates: 11.00ms, Linking: 0.00ms)
[INFO][2024/04/17 12:23:39 PM]  - Probabilities (Link: 1.00000, Lost: 1.00000)
[INFO][2024/04/17 12:23:39 PM]  - Stats (Active: 495, Lost: 13440, Conflicts resolved: 1978)
[INFO

## Merging objects and tracking information

In [9]:
trackDataAll = pd.DataFrame(data,columns=['track_id','t','x','y'])
trackDataAll['parent'] = properties['parent']
trackDataAll['generation'] = properties['generation']
trackDataAll['root'] = properties['root']

In [10]:
len(trackDataAll)

80233

In [11]:
allData = pd.merge(left=data_df,right=trackDataAll,left_on=['centroid-0','centroid-1','t'],right_on=['x','y','t'],how='left')

In [12]:
print(f'Number of all objects: {len(allData)}')

Number of all objects: 75384


In [13]:
# check how many objects doesn't have a track_id
test = np.sum(allData.track_id!=allData.track_id)
print(f'Number of objects without track_id: {test}')

Number of objects without track_id: 0


## Be careful!!!

In [ ]:
# consider removing
#allData = allData.loc[allData.track_id==allData.track_id,:]
#print(f'Number of all objects: {len(allData)}')

## Define promising tracks

This part is manual at the moment.

In [14]:
my_tracks = set(allData.track_id)
print(len(my_tracks))

6041


In [15]:
allData['accepted'] = False
allData['rejected'] = False
allData['promise'] = False

# Assuming track_lengths_to_test is a list of track lengths you want to test
track_lengths_to_test = [10, 20, 30, 40, 50, 70, 90, 100, 110, 120]  # Example thresholds

# Initialize a dictionary to hold the count of tracks exceeding each length
tracks_exceeding_length = {length: 0 for length in track_lengths_to_test}

for track in set(allData.track_id):
    # Prepare signals for this track
    sel_signal = allData.loc[allData.track_id == track, ['t', 'mean_intensity-0_nuc', 'mean_intensity-0_ring']]
    sel_signal.sort_values(by='t', inplace=True)
    
    track_len = len(sel_signal)
    
    # Update the count for each threshold exceeded
    for length in track_lengths_to_test:
        if track_len > length:
            tracks_exceeding_length[length] += 1

# Report on the number of tracks exceeding each length threshold
for length, count in tracks_exceeding_length.items():
    print(f"Number of tracks longer than {length}: {count}")


Number of tracks longer than 10: 1361
Number of tracks longer than 20: 802
Number of tracks longer than 30: 565
Number of tracks longer than 40: 405
Number of tracks longer than 50: 335
Number of tracks longer than 70: 232
Number of tracks longer than 90: 172
Number of tracks longer than 100: 157
Number of tracks longer than 110: 147
Number of tracks longer than 120: 134


In [16]:
specific_length = 50  # Example specific length
promise_list = []

for track in set(allData.track_id):
    # Prepare signals for this track
    sel_signal = allData[allData.track_id == track]
    sel_signal.sort_values(by='t', inplace=True)  # Ensure chronological order
    
    if len(sel_signal) > specific_length:
        allData.loc[allData.track_id == track, 'promise'] = True
        promise_list.append(track)

print(f"Number of promising tracks longer than {specific_length}: {len(promise_list)}")


C:\Users\gases\AppData\Local\Temp\ipykernel_11896\2312636104.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sel_signal.sort_values(by='t', inplace=True)  # Ensure chronological order


Number of promising tracks longer than 50: 335


In [ ]:
#Old method of setting promising tracks
'''
allData['accepted'] = False
allData['rejected'] = False
allData['promise'] = False

# mark tracks longer than 100 as promising
tracks_set = set(allData.track_id)

track_len=[]
promise_list = []
for track in tracks_set:
    
    # prepare signals for this track
    sel_signal = allData.loc[allData.track_id == track,['t','mean_intensity-1_nuc','mean_intensity-1_ring']]
    sel_signal.sort_values(by='t',inplace=True)
    sel_mean = sel_signal.rolling(9,min_periods = 9,center=True).mean()
    
    # test - length
    track_test = len(sel_signal)>50
    
    track_len.append(len(sel_signal))
    
    # test - DHB presence
    dhb_test = np.sum(sel_mean['mean_intensity-1_nuc'] > (sel_mean['mean_intensity-1_ring']+100)) > 10
    
    if (track_test and dhb_test):
        
        promise_list.append(track)
        
        allData.loc[allData.track_id==track,'promise'] = True
'''

In [17]:
len(promise_list)

335

## Create columns for requested annotations

In [18]:
# get info about the tags (for annotating points on the tracks)
flag_list = inp_f.read_flags(info_lines,df=allData)

for flag in flag_list:
    
    allData[flag['flag_column']]=False

In [19]:
# save df
allData.to_pickle(os.path.join(df_dir,df_name))
#allData.to_csv(os.path.join(df_dir,df_name.replace('pkl','csv')),index=False)